# Tier 2 AI-Terminal Protocol – Improvement Plan

This notebook captures implementation-focused experiments that address the key improvement areas: cross-platform support, dependency checks, richer documentation, resilient state management, and tmux recovery automation.

## 1. Environment Pre-flight Checks with Bash

These bash snippets detect the host platform, verify required binaries (tmux, inotifywait, fswatch, powershell), and emit structured exit codes so the installer can branch on missing dependencies.

In [ ]:
#!/usr/bin/env bash
set -euo pipefail

OS_NAME="$(uname -s 2>/dev/null || echo unknown)"
case "${OS_NAME}" in
    Linux)
        PLATFORM="linux"
        WATCH_PROBES=("inotifywait")
        ;;
    Darwin)
        PLATFORM="macos"
        WATCH_PROBES=("fswatch" "watchman")
        ;;
    MINGW*|MSYS*|CYGWIN*)
        PLATFORM="windows"
        WATCH_PROBES=("powershell.exe")
        ;;
    *)
        PLATFORM="unknown"
        WATCH_PROBES=()
        ;;
 esac

missing=()
check_binary() {
    if ! command -v "$1" >/dev/null 2>&1; then
        missing+=("$1")
    fi
}

check_binary tmux
check_binary sqlite3
for probe in "${WATCH_PROBES[@]}"; do
    check_binary "$probe"
done

if [ "${#missing[@]}" -gt 0 ]; then
    printf '{"status":"fail","platform":"%s","missing":["%s"]}\n' "$PLATFORM" "${missing[*]}"
    exit 2
fi

printf '{"status":"ok","platform":"%s"}\n' "$PLATFORM"

## 2. Cross-Platform File Watcher Adapter

The following Python module auto-selects an event watcher implementation (inotifywait, fswatch, or PowerShell). A tiny harness validates uniform event dispatch so higher-level scripts stay portable.

In [ ]:
import os
import platform
import shutil
from dataclasses import dataclass
from typing import Callable, Iterable, Optional

@dataclass
class WatchCommand:
    runner: str
    args: list[str]

@dataclass
class WatchEvent:
    path: str
    kind: str

class WatcherNotAvailable(RuntimeError):
    pass

class WatcherFactory:
    def __init__(self) -> None:
        self._system = platform.system()

    def build(self, directory: str, recursive: bool = True) -> WatchCommand:
        candidates: list[Callable[[str, bool], Optional[WatchCommand]]] = [
            self._inotify_command,
            self._fswatch_command,
            self._powershell_command,
        ]
        for builder in candidates:
            command = builder(directory, recursive)
            if command:
                return command
        raise WatcherNotAvailable(f"No watcher available on {self._system}")

    def _inotify_command(self, directory: str, recursive: bool) -> Optional[WatchCommand]:
        if shutil.which("inotifywait") and self._system == "Linux":
            args = ["inotifywait", "-m", "-e", "modify", directory]
            if recursive:
                args.insert(2, "-r")
            return WatchCommand("bash", args)
        return None

    def _fswatch_command(self, directory: str, recursive: bool) -> Optional[WatchCommand]:
        if shutil.which("fswatch") and self._system == "Darwin":
            args = ["fswatch", "-0"]
            if recursive:
                args.extend(["-r"])
            args.append(directory)
            return WatchCommand("bash", args)
        return None

    def _powershell_command(self, directory: str, recursive: bool) -> Optional[WatchCommand]:
        if shutil.which("powershell") and self._system == "Windows":
            script = (
                "Register-ObjectEvent -InputObject (New-Object IO.FileSystemWatcher"
                f" -ArgumentList '{directory}', '*') -EventName Changed"
            )
            if recursive:
                script = script.replace("FileSystemWatcher", "FileSystemWatcher -IncludeSubdirectories $true")
            return WatchCommand("powershell", ["powershell", "-NoLogo", "-NoProfile", "-Command", script])
        return None


def normalize_output(raw: Iterable[str]) -> list[WatchEvent]:
    events: list[WatchEvent] = []
    for line in raw:
        parts = line.split()
        if len(parts) < 2:
            continue
        events.append(WatchEvent(path=parts[0], kind=parts[1]))
    return events

In [ ]:
import unittest
from unittest.mock import patch

class WatcherFactoryTests(unittest.TestCase):
    def test_selects_inotify_on_linux(self):
        factory = WatcherFactory()
        with patch("platform.system", return_value="Linux"), patch("shutil.which", return_value="/usr/bin/inotifywait"):
            factory._system = "Linux"
            cmd = factory.build("/tmp")
            self.assertIn("inotifywait", cmd.args[0])

    def test_raises_when_missing(self):
        factory = WatcherFactory()
        with patch("platform.system", return_value="Plan9"), patch("shutil.which", return_value=None):
            factory._system = "Plan9"
            with self.assertRaises(WatcherNotAvailable):
                factory.build("/tmp")

unittest.TextTestRunner(verbosity=2).run(unittest.defaultTestLoader.loadTestsFromTestCase(WatcherFactoryTests))

## 3. SQLite State Backend Prototype

These cells sketch out a SQLite-backed state store (agent sessions, events, panes) and compare file-based writes versus batched inserts to gauge scalability trade-offs.

In [ ]:
import sqlite3
from pathlib import Path
from time import perf_counter

db_path = Path("./liku_state_prototype.db")
if db_path.exists():
    db_path.unlink()

conn = sqlite3.connect(db_path)
cur = conn.cursor()
cur.executescript(
    """
    CREATE TABLE agent_session (
        id INTEGER PRIMARY KEY,
        agent_name TEXT NOT NULL,
        session_key TEXT NOT NULL,
        terminal_id TEXT,
        created_at TEXT DEFAULT CURRENT_TIMESTAMP
    );
    CREATE TABLE tmux_pane (
        id INTEGER PRIMARY KEY,
        session_key TEXT NOT NULL,
        terminal_id TEXT NOT NULL,
        status TEXT NOT NULL,
        last_command TEXT,
        updated_at TEXT DEFAULT CURRENT_TIMESTAMP
    );
    CREATE TABLE event_log (
        id INTEGER PRIMARY KEY,
        event_type TEXT NOT NULL,
        payload TEXT NOT NULL,
        created_at TEXT DEFAULT CURRENT_TIMESTAMP
    );
    CREATE INDEX idx_event_type ON event_log(event_type);
    """
)
conn.commit()
print("Schema created", db_path)

In [ ]:
import json

sample_agents = [
    ("build-agent", "session-1", "liku:0.0"),
    ("test-agent", "session-1", "liku:0.1"),
]
cur.executemany(
    "INSERT INTO agent_session (agent_name, session_key, terminal_id) VALUES (?, ?, ?)",
    sample_agents,
)

payload = json.dumps({"event": "agent.spawn", "agent": "build-agent"})
cur.execute(
    "INSERT INTO event_log (event_type, payload) VALUES (?, ?)",
    ("agent.spawn", payload),
)

conn.commit()

cur.execute("SELECT agent_name, terminal_id FROM agent_session")
print("Agents:", cur.fetchall())
cur.execute("SELECT event_type, payload FROM event_log")
print("Events:", cur.fetchall())

In [ ]:
import tempfile
import time

records = 500
payload = json.dumps({"event": "heartbeat", "tty": "pts/1"})

start = perf_counter()
with tempfile.TemporaryDirectory() as tmpdir:
    for i in range(records):
        with open(Path(tmpdir) / f"event-{i}.json", "w", encoding="utf-8") as fh:
            fh.write(payload)
file_elapsed = perf_counter() - start

start = perf_counter()
cur.executemany(
    "INSERT INTO event_log (event_type, payload) VALUES (?, ?)",
    [("heartbeat", payload)] * records,
)
conn.commit()
sqlite_elapsed = perf_counter() - start

print(f"File writes:  {file_elapsed:.4f}s for {records} events")
print(f"SQLite batch: {sqlite_elapsed:.4f}s for {records} events")

## 4. Fault-Tolerant tmux Session Recovery

This shell toolkit sweeps for orphaned panes, rebuilds expected LIKU sessions, and emits audit logs that Bookkeeper can replay during recovery drills.

In [ ]:
#!/usr/bin/env bash
set -euo pipefail

tmux list-sessions >/dev/null 2>&1 || {
    echo "{}"; exit 0;
}

RECOVERY_LOG="./tmux-recovery.log"
EXPECTED_SESSION="$(liku_session=${LIKU_SESSION:-$(whoami)-liku}; echo "$liku_session")"

scan_orphans() {
    tmux list-panes -a -F '#S:#I.#P #{pane_dead}' | awk '$2=="1" {print $1}'
}

restart_orphan() {
    local pane_id="$1"
    tmux kill-pane -t "$pane_id" || true
    tmux new-window -t "$EXPECTED_SESSION" -n "recovered" -d
    printf '{"event":"tmux.recover","pane":"%s"}\n' "$pane_id" >> "$RECOVERY_LOG"
}

mapfile -t ORPHANS < <(scan_orphans)
if [ "${#ORPHANS[@]}" -eq 0 ]; then
    printf '{"status":"clean"}\n'
else
    for pane in "${ORPHANS[@]}"; do
        restart_orphan "$pane"
    done
    printf '{"status":"recovered","count":%d}\n' "${#ORPHANS[@]}"
fi

## 5. Documentation Scaffolding via Scripts

Automate contributor guides by introspecting project metadata and emitting Markdown with tables of contents and cross-links so the docs stay in sync with the codebase.

In [ ]:
import textwrap
from pathlib import Path

PROJECT_ROOT = Path("..")
AGENT_DIR = PROJECT_ROOT / "agents"
CORE_DIR = PROJECT_ROOT / "core"
DOC_OUTPUT = Path("./generated-guides.md")

sections = []
sections.append("# LIKU Contribution Guide\n")
sections.append("## Agents Overview\n")
for agent in sorted(p.name for p in AGENT_DIR.iterdir() if p.is_dir() and not p.name.startswith(".")):
    sections.append(f"- [{agent}](agents/{agent})")
sections.append("\n## Core Services\n")
for script in sorted(p.name for p in CORE_DIR.glob("*.sh")):
    sections.append(f"- {script}")
sections.append("\n## Quick Start Tasks\n")
sections.append("1. Run `./install.sh`\n2. Launch `liku bookkeeper`\n3. Start `liku exec --window general -- <cmd>`\n")

DOC_OUTPUT.write_text("\n".join(sections), encoding="utf-8")
print("Guide generated:", DOC_OUTPUT.resolve())